Training Notebook: 
https://www.kaggle.com/maunish/chaii-pytorch-train-xlmroberta-large

In [ ]:
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq

In [ ]:
import os
import gc
import sys
import math
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from functools import partial
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForQuestionAnswering,AutoModel,
                          AutoConfig,AdamW, get_cosine_schedule_with_warmup)

In [ ]:
train_data = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
test_data = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
sample = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')

config = {'max_length':384,
          'doc_stride':128,
          'max_answer_length':30,
          
          'lr':1e-5,
          'wd':0.01,
          'epochs':1,
          'nfolds':5,
          'batch_size':32,
          'num_workers':4,
          'seed':1000}

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

In [ ]:
def prepare_validation_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    # ref: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
def postprocess_qa_predictions(
    examples, tokenizer, features, raw_predictions, n_best_size=20, max_answer_length=30, squad_v2=False
):
    # ref: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(examples):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
class ChaiiDataset:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):            
        return {"input_ids": torch.tensor(self.data[idx]["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(self.data[idx]["attention_mask"], dtype=torch.long)}

In [ ]:
class Model(nn.Module):
    def __init__(self,model_name):
        super(Model,self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.roberta = AutoModel.from_pretrained(model_name,config=config)
        self.roberta.pooler = nn.Identity()
        self.linear = nn.Linear(self.config.hidden_size,2)
        
    def loss_fn(self,start_logits,end_logits,start_positions,end_positions):
        if len(start_positions.size()) > 1:
            start_positions = start_positions.squeeze(-1)
        if len(end_positions.size()) > 1:
            end_positions = end_positions.squeeze(-1)

        ignored_index = start_logits.size(1)
        start_positions = start_positions.clamp(0, ignored_index)
        end_positions = end_positions.clamp(0, ignored_index)
        loss_fct = nn.CrossEntropyLoss(ignore_index=ignored_index)
        start_loss = loss_fct(start_logits, start_positions)
        end_loss = loss_fct(end_logits, end_positions)
        total_loss = 0.75 * start_loss + end_loss * 0.25
        return total_loss
    
    def forward(self,**xb):
        x = self.roberta(input_ids=xb['input_ids'],attention_mask=xb['attention_mask'])[0]
        x = self.linear(x)
        
        start_logits,end_logits = x.split(1,dim=-1)
        start_logits,end_logits = start_logits.squeeze(-1).contiguous(),end_logits.squeeze(-1).contiguous()
            
        return (start_logits,end_logits)

In [ ]:
def get_prediction(df,model_paths,device='cuda'):
    start_logits = list()
    end_logits = list()
    
    for path,model_name in model_paths:
#         model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        model = Model(model_name)
        model.eval()
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        pad_on_right = tokenizer.padding_side == 'right'

        for f in range(config['nfolds']):
            model.load_state_dict(torch.load(path.format(f),map_location=device))
            model.to(device)
            model.eval()

            test_dataset = Dataset.from_pandas(df)
            test_features = test_dataset.map(
                            partial(
                                prepare_validation_features, 
                                tokenizer=tokenizer,
                                pad_on_right=pad_on_right, 
                                max_length=config['max_length'],
                                doc_stride=config['doc_stride']
                            ),
                            batched=True,
                            remove_columns=test_dataset.column_names)

            test_feats_small = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

            test_ds = ChaiiDataset(test_feats_small)
            test_dl = DataLoader(test_ds,
                                batch_size = config["batch_size"],
                                num_workers = config['num_workers'],
                                shuffle=False,
                                pin_memory=True,
                                drop_last=False)

            with torch.no_grad():
                pred = list()
                start_logit = list()
                end_logit = list()
                for i, inputs in enumerate(test_dl):
                    inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
                    outputs = model(**inputs)
                    start = outputs[0].detach().cpu().numpy().tolist()
                    end = outputs[1].detach().cpu().numpy().tolist()
                    start_logit.extend(start)
                    end_logit.extend(end)

            start_logits.append(start_logit)
            end_logits.append(end_logit)

    torch.cuda.empty_cache()
    start_logits, end_logits = np.mean(start_logits,axis=0), np.mean(end_logits,axis=0)

    fin_preds = postprocess_qa_predictions(test_dataset, tokenizer, test_features, (start_logits, end_logits))
    return fin_preds

In [ ]:
model_paths = [
    ('../input/chaii-pytorch-xlmroberta-large/model{0}/model{0}.bin','../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'),
]

In [ ]:
predictions = get_prediction(test_data,model_paths)

In [ ]:
test_data['PredictionString'] = test_data['id'].map(predictions)

In [ ]:
bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

tamil_ad = "கி.பி"
tamil_bc = "கி.மு"
tamil_km = "கி.மீ"
hindi_ad = "ई"
hindi_bc = "ई.पू"

cleaned_preds = []
for pred, context in test_data[["PredictionString", "context"]].to_numpy():
    if pred == "":
        cleaned_preds.append(pred)
        continue
    while any([pred.startswith(y) for y in bad_starts]):
        pred = pred[1:]
    while any([pred.endswith(y) for y in bad_endings]):
        if pred.endswith("..."):
            pred = pred[:-3]
        else:
            pred = pred[:-1]
    
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."

    cleaned_preds.append(pred)

test_data["PredictionString"] = cleaned_preds

In [ ]:
test_data[['id', 'PredictionString']].to_csv('submission.csv', index=False)

In [ ]:
test_data